# Setting up the computational model
To simulate the motion of molecules according to the kinetic theory of gases, we need to set up a dynamic n-body simulation with collisions. Note that the objective of this article is to produce a simulation for pedagogical purposes. Therefore, the code is set up to maximize understanding, not execution speed.

## Defining molecular properties
We start by importing all the required modules.

In [1]:
import os
import sys
import time
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

First, we will define a class called Molecule. Objects of this class will store properties such as the masses, positions and velocities of molecules in the simulation. We also define an attribute called color, that may be used to distinguish between different gases and set the color of the points in the animation.

In [ ]:
class Molecule(object):
    #Constructor
    def __init__(self,molecule,position,velocity,mass,color="black"):
        self.molecule=molecule
        self.position=np.array([x_i for x_i in position])
        self.velocity=np.array([v_i for v_i in velocity])
        self.mass=mass
        self.color=color
       
    #Setters for position, velocity, mass and color
    def set_position(self,position):
        self.position=np.array([x_i for x_i in position])
        
    def set_velocity(self,velocity):
        self.velocity=np.array([v_i for v_i in velocity])
        
    def set_color(self,color):
        self.color=color
    
    def set_mass(self,mass):
        self.mass=mass
        
    #Getters for position, velocity, mass and color
    def get_position(self):
        return self.position
    
    def get_velocity(self):
        return self.velocity
        
    def get_color(self):
        return self.color
        
    def get_mass(self):
        return self.mass